In [1]:
import numpy as np
from scipy.stats import norm

In [2]:
# Function to generate synthetic data
def generate_data(n, m, d, clusters, noise_var):
    X = []
    y = []
    w_true = []
    for c in range(clusters):
        w_c = np.random.uniform(-5, 5, d)  # Generate weight vector w_c of size d using uniform distribution
        w_true.append(w_c)
        for i in range(n // clusters):
            X_i = np.random.normal(0, 1, (m, d)) # Generate matrix X_i of size (m, d) using normal / gaussian distribution
            y_i = X_i @ w_c + np.random.normal(0, noise_var, m)  # Generate vector y_i of size m using the formula: y_i = X_i * wc + ϵ_i
            X.append(X_i)
            y.append(y_i)
    return X, y, w_true

In [3]:
# Personalized Federated Learning (Algorithm 1)
def personalized_federated_learning(X, y, learning_rate=0.01, iterations=100, candidate_count=20):
    n = len(X)  # Number of data generators
    d = X[0].shape[1]  # Number of features per data generator
    
    #Step 1: Initialize parameters for data generator 1 (w_hat := 0)
    w_est = np.zeros(d)  

    # Step 2: Iterate for R iterations
    for _ in range(iterations):  
        
        # Step 3: Randomly choose a subset of candidates (C) excluding data generator
        candidates = np.random.choice(range(1, n), size=candidate_count, replace=False)  
        
        best_candidate = None 
        max_reward = float('-inf')  
        
        current_loss = np.mean((y[0] - X[0] @ w_est) ** 2)

        for i in candidates:
            # Step 4: Compute gradient for each candidate i
            grad = -X[i].T @ (y[i] - X[i] @ w_est)
            w_temp = w_est - learning_rate * grad

            # Step 5: Calculate the loss for each candidate
            updated_loss = np.mean((y[0] - X[0] @ w_temp) ** 2)

            # Step 6: Determine the best candidate based on maximum reward
            reward = current_loss - updated_loss

            # Determine the best candidate based on maximum reward
            if reward > max_reward:
                max_reward = reward
                best_candidate = i

        # Step 7: Update w_est using the best candidate's gradient if found
        if best_candidate is not None:
            grad = -X[best_candidate].T @ (y[best_candidate] - X[best_candidate] @ w_est)
            w_est -= learning_rate * grad

    return w_est

In [4]:
# Expectation-Maximization
def personalized_expectation_maximization(X, y, clusters, iterations=20, learning_rate=0.01, step_size=0.1, epsilon=0.3, beta=0.6, neighbors_per_iteration=3, stability_epsilon=1e-10):
    n = len(X)
    d = X[0].shape[1]

    # Initialize model parameters (Phi) and priors (Pi)
    Phi = np.random.randn(clusters, d)  # Model parameters for each cluster
    Pi = np.ones((n, clusters)) / clusters  # Uniform initial priors (representing mixture weights)
    L_hat = np.zeros((n, clusters))  # Exponential moving average of losses

    for t in range(iterations):
        # E-step: Compute the posterior probabilities w_ij for each client
        W = np.zeros((n, clusters))  # Weight matrix representing collaboration responsibilities
        
        # Step 1: Clients independently compute their own posteriors using neighbor selection
        for i in range(n):
            # ε-greedy neighbor selection: balance between exploration and exploitation
            available_neighbors = clusters if clusters < neighbors_per_iteration else neighbors_per_iteration
            if np.random.rand() < epsilon:
                # Exploration: choose random neighbors
                neighbors = np.random.choice(clusters, min(available_neighbors, clusters), replace=False)
            else:
                # Exploitation: choose neighbors with highest priors (most probable collaborators)
                neighbors = np.argsort(Pi[i])[-min(available_neighbors, clusters):]

            # Compute posterior responsibilities for client i using only selected neighbors
            log_probs = np.full(clusters, -np.inf)  # Default to very low log probability for non-neighbors
            
            for j in neighbors:
                # Calculate the current loss and update the exponential moving average
                loss = np.sum((y[i] - X[i] @ Phi[j]) ** 2)
                L_hat[i, j] = (1 - beta) * L_hat[i, j] + beta * loss  # Update exponential moving average of loss

                # Calculate log prior + negative average loss for posterior computation
                log_probs[j] = np.log(Pi[i, j] + stability_epsilon) - L_hat[i, j]  # Adding stability_epsilon to prevent log(0)

            # Normalize using softmax to get posterior probabilities for selected neighbors
            max_log_prob = np.max(log_probs[neighbors])  # Numerical stability for softmax
            exp_probs = np.exp(log_probs[neighbors] - max_log_prob)
            W[i, neighbors] = exp_probs / np.sum(exp_probs)  # Normalize to get probabilities for selected neighbors

        # Sampling latent variables (z_i) from the categorical distribution represented by W
        Z = np.zeros((n,), dtype=int)  # Latent assignments for each data point
        for i in range(n):
            # Sampling z_i for each data point using the computed posterior probabilities over selected neighbors
            Z[i] = np.random.choice(clusters, p=W[i, :])

        # Step 2: Decentralized parameter updates by direct gradient sharing between clients
        new_Phi = np.copy(Phi)

        # Each client collects gradients from neighbors and performs independent updates
        for i in range(n):
            # Neighbor selection for gradient exchange using ε-greedy strategy
            available_neighbors = n if n < neighbors_per_iteration else neighbors_per_iteration
            if np.random.rand() < epsilon:
                neighbors = np.random.choice(range(n), min(available_neighbors, n), replace=False)
            else:
                neighbors = np.argsort(Pi[i])[-min(available_neighbors, n):]

            grad_accumulator = np.zeros(d)
            weight_accumulator = 0

            # Receive gradients from selected neighbors and aggregate them
            for neighbor in neighbors:
                cluster = Z[neighbor]
                X_assigned = X[neighbor]
                Y_assigned = y[neighbor]

                # Compute residual and gradient
                residual = X_assigned.dot(Phi[cluster]) - Y_assigned
                gradient = X_assigned.T.dot(residual)

                # Weight gradient by the posterior responsibility
                weight = W[neighbor, cluster]
                grad_accumulator += weight * gradient
                weight_accumulator += weight * X_assigned.shape[0]

            # Independent parameter update by client i for cluster Z[i]
            cluster = Z[i]
            if weight_accumulator > 0:
                grad_update = grad_accumulator / weight_accumulator
                new_Phi[cluster] -= step_size * learning_rate * grad_update  # Small gradient update

        # Step 3: Update priors dynamically based on shared information
        # Each client independently updates its priors based on neighbors' contributions
        for i in range(n):
            neighbors = np.argsort(Pi[i])[-min(neighbors_per_iteration, clusters):]
            weighted_sum = np.sum(W[i, neighbors])
            if weighted_sum > 0:
                Pi[i, neighbors] = W[i, neighbors] / weighted_sum  # Normalize to maintain valid distribution

        # Replace Phi with new_Phi after all updates
        Phi = new_Phi

    return Phi, Z

In [5]:
# Gibbs Sampling
def personalized_gibbs_sampling(X, y, clusters, iterations=1000, burn_in=100, candidate_count=20):
    n = len(X)
    d = X[0].shape[1]
    w_samples = np.zeros((iterations - burn_in, clusters, d))
    w_current = np.random.randn(clusters, d)

    for it in range(iterations):
        for c in range(clusters):
            # Choose a subset of candidates excluding data generator 1 (similar to personalized learning)
            candidates = np.random.choice(range(1, n), size=candidate_count, replace=False)

            # Compute weighted sum and total weight from candidates for updating the cluster parameter
            weighted_sum = np.zeros(d)
            total_weight = np.zeros((d, d))

            # Iterate over the selected candidates
            for i in candidates:
                weighted_sum += X[i].T @ y[i]
                total_weight += X[i].T @ X[i]

            # Add the contribution from data generator 1 to personalize the model
            weighted_sum += X[0].T @ y[0]
            total_weight += X[0].T @ X[0]

            # Sample new parameters from the conditional posterior distribution
            w_current[c] = np.random.multivariate_normal(np.linalg.inv(total_weight) @ weighted_sum, np.linalg.inv(total_weight))

        # Store the samples after burn-in period
        if it >= burn_in:
            w_samples[it - burn_in] = w_current

    # Return the mean of the samples as the personalized estimate for data generator 1
    return w_samples.mean(axis=0)

In [6]:
# Performance evaluation
def evaluate_performance(w_true, w_est):
    clusters = w_est.shape[0]
    return np.mean([np.linalg.norm(w_est[i % clusters] - w_true[i]) ** 2 for i in range(len(w_true))])

In [7]:
n, m, d, clusters = 100, 10, 5, 2
noise_var = 0.1
iterations = 10

w_pfl_avg = 0
w_em_avg = 0
w_gibbs_avg = 0

for _ in range(iterations):
    X, y, w_true = generate_data(n, m, d, clusters, noise_var)

    # Run Personalized Federated Learning
    w_pfl = personalized_federated_learning(X, y)

    # Run Updated Expectation-Maximization for Federated Learning with Discrete Latent Assignment (`zi`)
    w_em_federated, zi = personalized_expectation_maximization(X, y, clusters)

    # Run Gibbs Sampling
    w_gibbs = personalized_gibbs_sampling(X, y, clusters)

    # Evaluate the performance of each method
    pfl_performance = evaluate_performance(w_true, np.array([w_pfl]))
    em_federated_performance = evaluate_performance(w_true, w_em_federated)
    gibbs_performance = evaluate_performance(w_true, w_gibbs)
    
    w_pfl_avg += pfl_performance
    w_em_avg += em_federated_performance
    w_gibbs_avg += gibbs_performance

    # Display performance results
    print(f"Personalized Federated Learning Performance (MSE): {pfl_performance}")
    print(f"Federated EM Performance (MSE): {em_federated_performance}")
    print(f"Gibbs Sampling Performance (MSE): {gibbs_performance}")
    
    print(f"\n============================\n")
    

print(f"Average PFL Performance (MSE): {w_pfl_avg / 10}")
print(f"Average EM Performance (MSE): {w_em_avg / 10}")
print(f"Average Gibbs Sampling Performance (MSE): {w_gibbs_avg / 10}")

Personalized Federated Learning Performance (MSE): 50.25453628175528
Federated EM Performance (MSE): 22.6053920037701
Gibbs Sampling Performance (MSE): 25.1067633407318


Personalized Federated Learning Performance (MSE): 55.90809428111086
Federated EM Performance (MSE): 50.12128159463204
Gibbs Sampling Performance (MSE): 27.710867253945075


Personalized Federated Learning Performance (MSE): 29.50282810503096
Federated EM Performance (MSE): 35.597795718631566
Gibbs Sampling Performance (MSE): 14.498815964838201


Personalized Federated Learning Performance (MSE): 39.16617174200603
Federated EM Performance (MSE): 20.306525336052726
Gibbs Sampling Performance (MSE): 20.090412194319995


Personalized Federated Learning Performance (MSE): 48.18541941843612
Federated EM Performance (MSE): 20.363900931500975
Gibbs Sampling Performance (MSE): 24.74855252937033


Personalized Federated Learning Performance (MSE): 22.97202249748333
Federated EM Performance (MSE): 25.301980649923834
Gibbs Sampl